In [3]:
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
from IPython.display import display, HTML
import numpy as np
import folium
from geopy.geocoders import Nominatim
from geopy.distance import distance
from geopy.distance import geodesic
import os
import matplotlib.cm as cm
import matplotlib.colors as colors


In [4]:
DOWNLOAD_URL = "https://www.kaggle.com/himanshupoddar/zomato-bangalore-restaurants/download"
LINK_TO_KAGGLE_NOTEBOOK = "https://www.kaggle.com/animesh420/kernel103fd8fc20/edit"

In [6]:
# Reading zomato dataframe
# Download and unzip the csv file from the link above
# Or, see a working example in the kaggle url
zomato = pd.read_csv('./zomato.csv')
location_coordinates = dict()

FileNotFoundError: [Errno 2] File b'./zomato.csv' does not exist: b'./zomato.csv'

In [12]:
# Reading locations of each zomato circle ie neighborhood
zomato = zomato[zomato.location.map(lambda x: x is not np.nan)]
zomato['location'] = ['Koramangala' if 'koramangala' in x.lower() else x for x in zomato['location']]

import time


def outlier_handling(s):
    if s == "CV Raman Nagar":
        return "C V Raman Nagar"
    elif s == "Rammurthy Nagar":
        return "Ramamurthy Nagar"
    elif s == "Sadashiv Nagar":
        return "Sadashiva Nagar"
    elif s == "Bellandur":
        return "Bellandur Central"
    elif s == "Electronic City":
        return "Electronic City Phase 1"
    else:
        return s


location_coordinates = dict()
geolocator = Nominatim(user_agent="ny_explorer")

def get_location():
    exceptionRaised = False
    unique_locations = sorted(zomato['location'].unique())
    """
    To avoid following locations:
    1. North Bangalore 2. East Bangalore 3. South Bangalore 4. West Bangalore
    """ 
    unique_locations = [x for x in unique_locations if 'bangalore' not in x.lower()]
    i = 0
    zdf = pd.DataFrame()
    if 'zdf.csv' in os.listdir():
        zdf = pd.read_csv('./zdf.csv', index_col=0)

    while i < len(unique_locations):
        try:
            s = outlier_handling(unique_locations[i])

            if zdf is not None and s in zdf.index:
                print('Found {} loaded'.format(s))
                i += 1
                continue

            address = '{}, Bangalore'.format(s)

            location = geolocator.geocode(address)
            location_coordinates[s] = (location.latitude, location.longitude)
            zdf.loc[unique_locations[i], 'lats'] = location.latitude
            zdf.loc[unique_locations[i], 'longs'] = location.longitude
            print('Index = {}, Address = {} is located at lat = {}, lon = {}'.format(i, address, location.latitude,
                                                                                     location.longitude))
            i += 1
        except Exception as e:
            if type(e) == AttributeError:
                print(s, e)
                return
            time.sleep(5)

    return zdf

zdf = get_location() 
display(zdf.head())



Index = 0, Address = BTM, Bangalore is located at lat = 12.91127585, lon = 77.6045654343118
Index = 1, Address = Banashankari, Bangalore is located at lat = 12.9152208, lon = 77.573598
Index = 2, Address = Banaswadi, Bangalore is located at lat = 13.0141618, lon = 77.6518539
Index = 3, Address = Bannerghatta Road, Bangalore is located at lat = 12.9106394, lon = 77.6000462
Index = 4, Address = Basavanagudi, Bangalore is located at lat = 12.9417261, lon = 77.5755021
Index = 5, Address = Basaveshwara Nagar, Bangalore is located at lat = 12.9932236, lon = 77.5391579
Index = 6, Address = Bellandur Central, Bangalore is located at lat = 12.9245832, lon = 77.6735137
Index = 7, Address = Bommanahalli, Bangalore is located at lat = 13.3457317, lon = 77.4333346
Index = 8, Address = Brigade Road, Bangalore is located at lat = 12.9736132, lon = 77.6074716
Index = 9, Address = Brookefield, Bangalore is located at lat = 12.9668213, lon = 77.7168891
Index = 10, Address = C V Raman Nagar, Bangalore is

,lats,longs
BTM,12.911276,77.604565
Banashankari,12.915221,77.573598
Banaswadi,13.014162,77.651854
Bannerghatta Road,12.910639,77.600046
Basavanagudi,12.941726,77.575502


In [4]:
def get_distance(df, loc1, loc2):
    coords1 = df.loc[loc1, 'lats'], df.loc[loc1, 'longs']
    coords2 = df.loc[loc2, 'lats'], df.loc[loc2, 'longs']
    return geodesic(coords1, coords2).km

def getcolumns(n):
    f = ['1st', '2nd', '3rd']  
    return f if n < 4 else f  + ['{}th'.format(i) for i in range(4, n + 1)] 
def find_nearest(n=5):                                                                                                                                        
    columns = getcolumns(n)   
    unique_locations = zdf.index.unique()                                                             
    nearest = zdf.copy().dropna()                                                                              
    for location in unique_locations:                                                                 
        distanceSeries = pd.Series([get_distance(nearest, location, loc) for loc in unique_locations],    
                                   index=unique_locations)                                            
        nearest_five = distanceSeries.nsmallest(n=n + 1).index[1:]                                    
                                                                                                      
        for col, val in zip(columns, nearest_five):                                                   
            nearest.loc[location, col] = val                                                          
                                                                                                      
    return nearest.dropna()                                                                                    
nearest = find_nearest()
display(nearest[nearest.index.map(lambda x: x[0] == 'J')])

,lats,longs,1st,2nd,3rd,4th,5th
JP Nagar,12.907251,77.578271,Banashankari,Kanakapura Road,Kumaraswamy Layout,Bannerghatta Road,Jayanagar
Jakkur,13.078474,77.606894,Yelahanka,Residency Road,Sahakara Nagar,Nagawara,Hebbal
Jalahalli,13.046453,77.548380,New BEL Road,Yeshwantpur,Peenya,Sahakara Nagar,Hebbal
Jayanagar,12.929273,77.582423,Basavanagudi,Banashankari,JP Nagar,Wilson Garden,Bannerghatta Road
Jeevan Bhima Nagar,12.967807,77.656837,Church Street,Thippasandra,Old Airport Road,Indiranagar,Domlur


In [13]:
zdf.to_csv('./zdf.csv')

In [5]:
zom_loc = zomato.copy()
zom_loc = zom_loc[zom_loc.location.isin(zdf.index)]
dflist = []
zdf.dropna(axis=0)
for locs in zom_loc['location'].unique():
    df_to_change = zom_loc[zom_loc.location == locs]
    df_to_change.insert(0,'Lat',[zdf.loc[locs, 'lats']] * df_to_change.shape[0])
    df_to_change.insert(0,'Long',[zdf.loc[locs, 'longs']] * df_to_change.shape[0])
    dflist.append(df_to_change)
zom_loc = pd.concat(dflist)
print(zom_loc.location.unique()[:5])
print(zom_loc.rest_type.unique()[:5])

['Banashankari' 'Basavanagudi' 'Mysore Road' 'Jayanagar'
 'Kumaraswamy Layout']
['Casual Dining' 'Cafe, Casual Dining' 'Quick Bites' 'Casual Dining, Cafe'
 'Cafe']


In [6]:
zloc_vs_rt = zom_loc[zom_loc.rest_type.map(lambda x: type(x) == str)]
zloc_vs_rt = zloc_vs_rt[zloc_vs_rt.rate.map(lambda x: type(x) == str and '/' in x)]
zloc_vs_rt.rate = zloc_vs_rt.rate.apply(lambda x: float(x.split('/')[0]))
allRestTypes = sorted(zloc_vs_rt['rest_type'].unique())
allLocations = sorted(zloc_vs_rt['location'].unique())
recozom_rating = pd.DataFrame(columns=allRestTypes, index=allLocations).fillna(0)
recozom_frequency = pd.DataFrame(columns=allRestTypes, index=allLocations).fillna(0)
for i, locality in enumerate(allLocations):
    df1 = zloc_vs_rt[(zloc_vs_rt.location == locality)]

    for index, row in df1.iterrows():
        try:
            rating = row['rate']
            rest_type = row['rest_type']
            recozom_frequency.loc[locality, rest_type] += 1
            recozom_rating.loc[locality, rest_type] += rating / 5
        except Exception as e:
            print(e)


for index in recozom_rating.index:

    for col in recozom_rating.columns:
        if recozom_frequency.loc[index, col] != 0:
            recozom_rating.loc[index, col] /= recozom_frequency.loc[index, col]
            recozom_rating.loc[index, col] *= 5
recozom_frequency.dropna(inplace=True)
recozom_rating.dropna(inplace=True)
display(recozom_frequency.head())
display(recozom_rating.head())

,Bakery,"Bakery, Beverage Shop","Bakery, Cafe","Bakery, Dessert Parlor","Bakery, Food Court","Bakery, Quick Bites",Bar,"Bar, Casual Dining","Bar, Lounge","Bar, Pub","Bar, Quick Bites",Beverage Shop,"Beverage Shop, Cafe","Beverage Shop, Dessert Parlor","Beverage Shop, Quick Bites",Bhojanalya,Cafe,"Cafe, Bakery","Cafe, Bar","Cafe, Casual Dining","Cafe, Dessert Parlor","Cafe, Food Court","Cafe, Lounge","Cafe, Quick Bites",Casual Dining,"Casual Dining, Bar","Casual Dining, Cafe","Casual Dining, Irani Cafee","Casual Dining, Lounge","Casual Dining, Microbrewery","Casual Dining, Pub","Casual Dining, Quick Bites","Casual Dining, Sweet Shop",Club,"Club, Casual Dining",Confectionery,Delivery,Dessert Parlor,"Dessert Parlor, Bakery","Dessert Parlor, Beverage Shop","Dessert Parlor, Cafe","Dessert Parlor, Food Court","Dessert Parlor, Kiosk","Dessert Parlor, Quick Bites","Dessert Parlor, Sweet Shop",Dhaba,Fine Dining,"Fine Dining, Bar","Fine Dining, Lounge","Fine Dining, Microbrewery",Food Court,"Food Court, Beverage Shop","Food Court, Casual Dining","Food Court, Dessert Parlor","Food Court, Quick Bites",Kiosk,Lounge,"Lounge, Bar","Lounge, Cafe","Lounge, Casual Dining","Lounge, Microbrewery",Mess,Microbrewery,"Microbrewery, Bar","Microbrewery, Casual Dining","Microbrewery, Lounge","Microbrewery, Pub",Pub,"Pub, Bar","Pub, Cafe","Pub, Casual Dining","Pub, Microbrewery",Quick Bites,"Quick Bites, Bakery","Quick Bites, Beverage Shop","Quick Bites, Cafe","Quick Bites, Dessert Parlor","Quick Bites, Food Court","Quick Bites, Kiosk","Quick Bites, Meat Shop","Quick Bites, Mess","Quick Bites, Sweet Shop",Sweet Shop,"Sweet Shop, Quick Bites",Takeaway,"Takeaway, Delivery"
BTM,44,0,0,0,0,6,43,0,0,0,0,150,0,1,21,0,246,13,0,0,0,0,0,16,639,21,0,0,0,17,0,0,0,0,0,0,189,84,0,17,0,0,0,0,5,12,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,18,0,0,0,0,0,0,0,0,10,0,1958,0,8,0,17,0,0,0,0,18,37,26,22,261
Banashankari,17,0,0,2,0,0,21,4,0,0,0,5,0,3,7,0,85,0,0,11,0,0,0,8,135,0,5,0,0,0,0,0,0,0,0,0,23,43,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,2,0,0,0,0,0,6,0,0,0,0,330,2,0,3,0,0,0,0,0,0,15,5,3,5
Banaswadi,17,0,0,0,0,1,9,0,0,0,0,5,0,0,0,0,38,0,0,0,0,0,0,6,100,9,6,0,0,0,0,0,0,0,0,0,6,8,0,0,0,0,0,0,0,0,0,0,0,0,14,0,0,0,2,3,0,0,0,0,0,8,0,0,0,0,0,4,0,0,0,0,213,0,0,0,0,0,0,0,4,0,0,5,0,18
Bannerghatta Road,13,0,0,0,0,9,0,0,0,0,0,10,0,2,6,0,61,19,0,14,0,0,0,0,278,10,0,0,0,0,0,0,0,0,0,0,48,64,0,0,0,0,0,2,0,0,0,0,0,0,111,0,4,1,9,9,5,0,0,0,0,2,0,0,1,0,0,17,0,0,0,0,469,0,0,4,3,0,0,0,0,1,19,1,0,39
Basavanagudi,21,0,0,0,0,6,11,0,0,0,0,7,0,0,3,0,42,0,0,0,0,0,0,0,94,7,8,0,0,0,0,0,0,0,0,0,0,43,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,6,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,318,0,0,0,0,0,0,0,0,0,6,11,0,3


,Bakery,"Bakery, Beverage Shop","Bakery, Cafe","Bakery, Dessert Parlor","Bakery, Food Court","Bakery, Quick Bites",Bar,"Bar, Casual Dining","Bar, Lounge","Bar, Pub","Bar, Quick Bites",Beverage Shop,"Beverage Shop, Cafe","Beverage Shop, Dessert Parlor","Beverage Shop, Quick Bites",Bhojanalya,Cafe,"Cafe, Bakery","Cafe, Bar","Cafe, Casual Dining","Cafe, Dessert Parlor","Cafe, Food Court","Cafe, Lounge","Cafe, Quick Bites",Casual Dining,"Casual Dining, Bar","Casual Dining, Cafe","Casual Dining, Irani Cafee","Casual Dining, Lounge","Casual Dining, Microbrewery","Casual Dining, Pub","Casual Dining, Quick Bites","Casual Dining, Sweet Shop",Club,"Club, Casual Dining",Confectionery,Delivery,Dessert Parlor,"Dessert Parlor, Bakery","Dessert Parlor, Beverage Shop","Dessert Parlor, Cafe","Dessert Parlor, Food Court","Dessert Parlor, Kiosk","Dessert Parlor, Quick Bites","Dessert Parlor, Sweet Shop",Dhaba,Fine Dining,"Fine Dining, Bar","Fine Dining, Lounge","Fine Dining, Microbrewery",Food Court,"Food Court, Beverage Shop","Food Court, Casual Dining","Food Court, Dessert Parlor","Food Court, Quick Bites",Kiosk,Lounge,"Lounge, Bar","Lounge, Cafe","Lounge, Casual Dining","Lounge, Microbrewery",Mess,Microbrewery,"Microbrewery, Bar","Microbrewery, Casual Dining","Microbrewery, Lounge","Microbrewery, Pub",Pub,"Pub, Bar","Pub, Cafe","Pub, Casual Dining","Pub, Microbrewery",Quick Bites,"Quick Bites, Bakery","Quick Bites, Beverage Shop","Quick Bites, Cafe","Quick Bites, Dessert Parlor","Quick Bites, Food Court","Quick Bites, Kiosk","Quick Bites, Meat Shop","Quick Bites, Mess","Quick Bites, Sweet Shop",Sweet Shop,"Sweet Shop, Quick Bites",Takeaway,"Takeaway, Delivery"
BTM,3.731818,0.0,0.0,0.0,0.0,3.900000,3.430233,0.0,0.0,0.0,0.0,3.554000,0.0,3.800000,3.814286,0.0,3.639837,4.0,0.0,0.000000,0.0,0.0,0.0,3.55,3.614397,4.290476,0.0,0.0,0.0,4.2,0.0,0.0,0.0,0.0,0.0,0.0,3.532804,3.805952,0.0,4.094118,0.0,0.0,0.0,0.0,3.7,3.283333,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,3.775,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,4.277778,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.7,0.0,3.531767,0.0,3.25,0.0,3.558824,0.0,0.0,0.0,0.000,3.561111,3.535135,3.715385,3.477273,3.472414
Banashankari,3.694118,0.0,0.0,3.7,0.0,0.000000,3.085714,3.7,0.0,0.0,0.0,3.920000,0.0,3.733333,3.471429,0.0,3.760000,0.0,0.0,3.800000,0.0,0.0,0.0,3.80,3.732593,0.000000,4.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.682609,3.704651,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000,0.0,0.000000,3.80,0.0,0.0,0.0,0.0,3.900000,0.0,0.0,0.0,0.0,0.0,4.1,0.0,0.0,0.0,0.0,3.575758,3.4,0.00,3.6,0.000000,0.0,0.0,0.0,0.000,0.000000,3.720000,3.700000,3.133333,3.800000
Banaswadi,3.547059,0.0,0.0,0.0,0.0,3.400000,3.766667,0.0,0.0,0.0,0.0,3.460000,0.0,0.000000,0.000000,0.0,3.673684,0.0,0.0,0.000000,0.0,0.0,0.0,3.70,3.446000,3.444444,3.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.700000,3.625000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,3.657143,0.0,0.0,0.000,3.4,3.166667,0.00,0.0,0.0,0.0,0.0,3.662500,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,3.443662,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,3.425,0.000000,0.000000,3.800000,0.000000,3.444444
Bannerghatta Road,3.484615,0.0,0.0,0.0,0.0,3.066667,0.000000,0.0,0.0,0.0,0.0,3.500000,0.0,3.800000,3.950000,0.0,3.772131,3.8,0.0,3.428571,0.0,0.0,0.0,0.00,3.478777,4.340000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.464583,3.654688,0.0,0.000000,0.0,0.0,0.0,3.8,0.0,0.000000,0.0,0.0,0.0,0.0,3.409009,0.0,3.4,3.400,3.0,3.844444,3.78,0.0,0.0,0.0,0.0,3.700000,0.0,0.0,4.4,0.0,0.0,4.4,0.0,0.0,0.0,0.0,3.451173,0.0,0.00,2.7,3.500000,0.0,0.0,0.0,0.000,3.100000,3.384211,3.400000,0.000000,3.617949
Basavanagudi,3.761905,0.0,0.0,0.0,0.0,3.533333,3.563636,0.0,0.0,0.0,0.0,3.828571,0.0,0.000000,3.200000,0.0,3.783333,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,3.747872,3.385714,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,3.744186,0.0,0.000000,0.0,0.0,0.0,0.0,3.3,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000,0.0,3.600000,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,2.7,0.0,0.0,0.0,0.0,3.647799

In [7]:
col = 'listed_in(type)'
allRestTypes = sorted(zloc_vs_rt[col].unique())
allLocations = sorted(zloc_vs_rt['location'].unique())
recozom_listin_frequency = pd.DataFrame(columns=allRestTypes, index=allLocations).fillna(0)
recozom_listin_votes = pd.DataFrame(columns=allRestTypes, index=allLocations).fillna(0)
for restType in allRestTypes:

    df = zloc_vs_rt[zloc_vs_rt[col] == restType]
    resttypecols = df['rest_type'].unique()
    unique_locations = df['location'].unique()
    for locality in unique_locations:
        reco_f = recozom_frequency.loc[locality, resttypecols]
        recozom_listin_frequency.loc[locality, restType] = sum(reco_f.values)
        
 
recozom_listin_frequency.dropna(inplace=True)
display(recozom_listin_frequency.head())


,Buffet,Cafes,Delivery,Desserts,Dine-out,Drinks & nightlife,Pubs and bars
BTM,2688,3133,3903,3792,3859,976,834
Banashankari,516,634,744,704,738,266,0
Banaswadi,0,391,476,440,476,160,101
Bannerghatta Road,905,1047,1227,1180,1228,386,230
Basavanagudi,442,521,595,573,595,158,0


In [ ]:
def check_name(location):
    if location not in recozom_listin_frequency.index:
        possibles = [x for x in recozom_listin_frequency.index if x.startswith(location[0])]
        possibles = ','.join(possibles)
        print(possibles)
        msg= """Please choose a correct location. 
        Locations starting from letter {} are: {}""".format(location[0],possibles)
        raise Exception(msg) 
    else:
        return True
def find_type_of_restaurant(location,num_nearest=5,bestOptionNum=2,displayOption='table'):
    getString = lambda x: ','.join([y for y in x if type(y) == str])
    col = 'listed_in(type)'
    # check if valid location is passed in
    assert check_name(location) is True
    
    # check nearby locations as specified
    n = num_nearest
    nearest = find_nearest(n)
    columns = getcolumns(n)
    nearby_places = list(nearest.loc[location, columns].values)
    print(nearby_places, location)
    locationsToConsider = [location] + nearby_places
    print('Searching for restaurant to open at {} and nearby places at {}'.format(location, getString(nearby_places)))
    
    # Show best options ( popular in neighborhood and not available at nearby location)
    allRestaurants = recozom_listin_frequency.loc[locationsToConsider]
    collist = list(allRestaurants.columns)
    print('Best to open:')
    for i, cols in enumerate(collist):
        series = allRestaurants.loc[:, cols].nsmallest(1)
        print('{}. {} at {}'.format(i,cols, series.index[0]))
    
    # Show details for restaurant sub types
    # Show details for option(Choose a number) 
    bestOption = bestOptionNum
    assert bestOption < len(collist), 'Input a number less than {}'.format(len(collist))
    bestOption = collist[bestOption]
    print('Showing sub restaurant types for chosen option {}'.format(bestOption))

    subtypes = sorted(zloc_vs_rt[zloc_vs_rt[col] == bestOption]['rest_type'].unique())
    if len(subtypes) == 0:
        print('Sorry no sub type found.')
    else:
        print('Subtypes found successfully.')
    subRestaurants_freq = recozom_frequency.loc[locationsToConsider, subtypes].dropna()
    subRestaurants_rating = recozom_rating.loc[locationsToConsider, subtypes].dropna()
    bestSubTypes = subRestaurants_rating.idxmax(axis=1)
    geolocator = Nominatim(user_agent="ny_explorer")
    # Display option :: Enter 'log' to see a log or 'table' to see a table.
    option = displayOption
    
    if option == 'log':
 
        # Display information
        locationsToOpen = subRestaurants_freq.loc[:, bestSubTypes.values].idxmin()
        print('\n### Recommendations ### ::')
        for subrestType, location in locationsToOpen.items():
            print('.'*25)
            print('*** Recommend to open {}  at {} ***'.format(subrestType, location))
            best_location = bestSubTypes.index[np.where(bestSubTypes == subrestType)][0]
            dfn = zloc_vs_rt[(zloc_vs_rt.rest_type==subrestType) & (zloc_vs_rt.location==best_location)]
            dfn = dfn[dfn.rate == dfn.rate.max()]
            cols_to_disp = ['rate','name','cuisines','dish_liked','approx_cost(for two people)','address']
            dfn = dfn.loc[:, cols_to_disp].sample()


            for index, row in dfn.iterrows():

                print('Example of Higest rated restaurant {} with metrics (located at {}):'.format(subrestType,best_location))
                print(' 1.Restaurant Name : {}'.format(row[cols_to_disp[1]]))
                print(' 2.Rate : {}'.format(row[cols_to_disp[0]]))
                print(' 3.Cusinies : {}'.format(row[cols_to_disp[2]]))
                print(' 4.Dish Liked : {}'.format(row[cols_to_disp[3]]))
                print(' 5.Cost for two : {}'.format(row[cols_to_disp[4]]))
                print(' 6.Address : {}'.format(row[cols_to_disp[-1]]))
    else:
        columns = ['Recommend to open(Subrest type)', 'location to open', 'example restaurant', 'location of example',
                   'rating','cuisines','dish_liked','cost for two','Address of Example']
        df_to_display = pd.DataFrame(columns=columns)
         
        # Display information
        locationsToOpen = subRestaurants_freq.loc[:, bestSubTypes.values].idxmin()
        print('\n### Recommendations ### ::')
        for i,(subrestType, location) in enumerate(locationsToOpen.items()):
#             print('.'*25)
#             print('*** Recommend to open {}  at {} ***'.format(subrestType, location))
            df_to_display.loc[i, columns[0]] = subrestType
            df_to_display.loc[i, columns[1]] = location
            
            best_location = bestSubTypes.index[np.where(bestSubTypes == subrestType)][0]
            dfn = zloc_vs_rt[(zloc_vs_rt.rest_type==subrestType) & (zloc_vs_rt.location==best_location)]
            dfn = dfn[dfn.rate == dfn.rate.max()]
            cols_to_disp = ['rate','name','cuisines','dish_liked','approx_cost(for two people)','address']
            dfn = dfn.loc[:, cols_to_disp].sample()


            for index, row in dfn.iterrows():

                df_to_display.loc[i,columns[2]] = row[cols_to_disp[1]]
                df_to_display.loc[i,columns[3]] = best_location
                df_to_display.loc[i,columns[4]] = row[cols_to_disp[0]]
                df_to_display.loc[i,columns[5]] = row[cols_to_disp[2]]
                df_to_display.loc[i,columns[6]] = row[cols_to_disp[3]]
                df_to_display.loc[i,columns[7]] = row[cols_to_disp[4]]
                df_to_display.loc[i,columns[8]] = row[cols_to_disp[-1]]

                
        df_to_display.insert(0,'Restaurant type',[bestOption]*df_to_display.shape[0])
        display(df_to_display)
                
        
find_type_of_restaurant('BTM')
    

In [ ]:
col = 'listed_in(type)'
# display(list(zloc_vs_rt.columns))
zloc = zloc_vs_rt.copy().loc[:, ['location', 'votes',col]]
zloc = zloc.groupby(['location', col])['votes'].sum().reset_index().pivot(index=col, columns='location', values='votes')
zvotes = zloc.fillna(0)
display(zvotes)

In [1]:
def check_restaurant_type(restaurant_type):
    if restaurant_type not in recozom_listin_frequency.columns:
        possibles = list(recozom_listin_frequency.columns)
        possibles = ','.join(possibles)
        msg= """Please choose a correct Restaurant type. 
        Available locations are: {}""".format(possibles)
        raise Exception(msg) 
    else:
        return True
def find_name_of_location(restaurant_type,min_req=5,nearest_neighborhood=6,min_popular=7):
    assert check_restaurant_type(restaurant_type) is True
    #Minimum number of recommendations required.
    n = min_req
    locationSeries = recozom_listin_frequency.loc[:, restaurant_type].sort_values(ascending=True)
    # Enter number of nearest neighborhoods to check for.
    k = nearest_neighborhood
    # Enter minimum number of popular restaurant locations to show.
    m = min_popular
    nearest = find_nearest(k)
    location_df = nearest.copy().loc[locationSeries.index].drop(['lats', 'longs'], 1)
    location_df['Total'] = np.zeros(location_df.shape[0])
    cols = location_df.columns
    for index, row in location_df.iterrows():
        total = 0
        for col in cols:
            try:
                val = recozom_listin_frequency.loc[row[col], restaurant_type]
            except Exception as e:
                val = 0
            location_df.loc[index,col] = val
    location_df['Total'] = location_df.sum(axis=1)
    location_df = location_df.sort_values(by=['Total'])
    location_df.insert(0,'self',locationSeries.values)
    
    print('Displaying data (top {}) where {} is least available in neighborhoods'.format(n,restaurant_type))
    display(location_df.head(n))
    
    print('Displaying most popular (top {}) destinations for restaurant type {} (by votes)'.format(m,restaurant_type))
    display(zvotes.loc[restaurant_type,].sort_values(ascending=False).to_frame().head(m))
    


find_name_of_location('Buffet') 


NameError: name 'recozom_listin_frequency' is not defined

In [ ]:
def pandas_df_to_markdown_table(df):
    from IPython.display import Markdown, display
    fmt = ['---' for i in range(len(df.columns))]
    df_fmt = pd.DataFrame([fmt], columns=df.columns)
    df_formatted = pd.concat([df_fmt, df])
    display(Markdown(df_formatted.to_csv(sep="|", index=False)))

pandas_df_to_markdown_table(infodf)